In [1]:
import cv2
import tensorflow as tf
import numpy as np

In [2]:
# 加载预训练模型
model = tf.keras.applications.MobileNetV2(weights='imagenet')

In [ ]:

# 这个的输出有问题，没办法画框框
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
        # 运行预测
        predictions = model.predict(image.reshape(1, 224, 224, 3))

        # 解析预测结果
        results = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=5)[0]
        for result in results:
            print(result[1], result[2])

        # 在图像上标注检测到的物体
        for result in results:
            label = result[1]
            score = result[2]
            if score > 0.3:
                (startX, startY, endX, endY) = (0, 0, 0, 0) # 需要根据模型输出调整矩形框位置和大小
                cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
                cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 显示带有标注的图像
        cv2.imshow('Object Detection', image)
        if cv2.waitKey(1) == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()

In [3]:
def draw_rectangles(frame, results):
    """
    为识别出的物体绘制矩形框

    Args：
        frame: 当前摄像头捕获的帧
        results: 识别出的物体列表，每个物体包含标签、置信度和边界框坐标信息
    """

    # 对识别出的物体根据置信度从高到低排序
    results = sorted(results, key=lambda x: x[2], reverse=True)

    # 创建一个字典来存储已经绘制过的边界框
    used_rects = {}

    for result in results:
        label, confidence, (startX, startY, endX, endY) = result

        # 遍历之前绘制过的边界框，如果有重叠的则忽略当前物体
        overlapping = False
        for rect, _ in used_rects.items():
            if startX <= rect[0] and startY <= rect[1] and endX >= rect[2] and endY >= rect[3]:
                overlapping = True
                break
            elif startX >= rect[0] and startY >= rect[1] and endX <= rect[2] and endY <= rect[3]:
                overlapping = True
                break

        # 如果没有重叠的边界框，则在图像上绘制矩形框并将其添加到已使用边界框字典中
        if not overlapping:
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            used_rects[(startX, startY, endX, endY)] = label

    return frame


In [4]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret:
        height, width, channels = frame.shape
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
        # 运行预测
        predictions = model.predict(image.reshape(1, 224, 224, 3))

        # 解析预测结果
        results = tf.keras.applications.mobilenet_v2.decode_predictions(
            predictions, top=5)[0]
        for result in results:
            print(result[1], result[2])

        # 一直到这里没有任何问题，预测的值是对的，我们只要解决如何画框框就可以了，我想是不是把他封装成一个函数

        a = draw_rectangles(frame, results)

        cv2.imshow('Object Detection', frame)
        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 1s 644ms/step
spatula 0.052499115
Band_Aid 0.039486133
beaker 0.025030084
apron 0.024298832
sweatshirt 0.021742288


TypeError: cannot unpack non-iterable numpy.float32 object

In [7]:
import cv2
import numpy as np

# 加载模型
net = cv2.dnn.readNetFromTensorflow('frozen_inference_graph.pb', 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt')

# 加载类别名称
with open('coco-labels-paper.txt', 'rt') as f:
    labels = f.read().rstrip('\n').split('\n')

# 捕捉本地摄像头
cap = cv2.VideoCapture(0)

while True:
    # 读取视频帧
    ret, frame = cap.read()
    
    # 创建输入blob
    blob = cv2.dnn.blobFromImage(frame, size=(320, 320), swapRB=True)
    
    # 设置输入数据
    net.setInput(blob)
    
    # 运行模型并预测类别和边界框
    detections = net.forward()
    
    # 遍历检测结果
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:
            class_id = int(detections[0, 0, i, 1])
            class_name = labels[class_id]
            
            # 计算边界框坐标
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            x1, y1, x2, y2 = box.astype(int)
            
            # 在图像上绘制矩形框和类别信息
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f'{class_name}: {confidence:.2f}'
            cv2.putText(frame, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # 显示图像
    cv2.imshow('Object Detection', frame)
    
    # 按q键退出
    if cv2.waitKey(1) == ord('q'):
        break

# 释放摄像头并关闭所有窗口
cap.release()
cv2.destroyAllWindows()
